In [ ]:
from IPython.display import display, Markdown

from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker
from docling.document_converter import InputFormat
from sentence_transformers import SentenceTransformer

import chromadb
from chromadb.config import Settings

In [ ]:
converter = DocumentConverter()
doc = converter.convert("data/BBB.pdf").document

chunker = HybridChunker(
    max_tokens=1000,
    overlap=200
)

chunks = list(chunker.chunk(doc))

[INFO] 2026-02-04 18:22:23,226 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-02-04 18:22:23,238 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\Nathan\Projecten\ocr\.venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2026-02-04 18:22:23,239 [RapidOCR] main.py:53: Using C:\Users\Nathan\Projecten\ocr\.venv\Lib\site-packages\rapidocr\models\ch_PP-OCRv4_det_infer.onnx
[INFO] 2026-02-04 18:22:23,296 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-02-04 18:22:23,305 [RapidOCR] download_file.py:60: File exists and is valid: C:\Users\Nathan\Projecten\ocr\.venv\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2026-02-04 18:22:23,305 [RapidOCR] main.py:53: Using C:\Users\Nathan\Projecten\ocr\.venv\Lib\site-packages\rapidocr\models\ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2026-02-04 18:22:23,337 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-02-04 18:22:23,353 [RapidOCR]

### Utils 

In [234]:
def find_page_number(chunk):
    page_numbers = [
        prov.page_no
        for item in chunk.meta.doc_items
        for prov in item.prov
        if prov.page_no is not None
    ]
    
    return (min(page_numbers) if page_numbers else None)

def find_headings(chunk):
    headings = chunk.meta.headings if chunk.meta.headings else ''
    return ", ".join(headings)

def find_pdf_name(chunk):
    return chunk.meta.origin.filename if chunk.meta.origin else None

def chunk_to_markdown(chunk_text: str) -> str:
    # convert a text string into a Docling Markdown document
    result = converter.convert_string(chunk_text, format=InputFormat.MD)
    return result.document.export_to_markdown()

### Create Vector store

In [142]:
def create_chromadb_collection(name="pdf_chunks", persist_dir="./chroma_db"):
    # Create a persistent client
    client = chromadb.PersistentClient(
        path=persist_dir,
        settings=Settings(
            persist_directory=persist_dir,
            is_persistent=True
        )
    )

    # Delete existing collection if exists
    if name in [col.name for col in client.list_collections()]:
        client.delete_collection(name)

    # Create the new collection
    collection = client.create_collection(name=name)
    return client, collection

### Create Embeddings & store in ChromaDB

In [143]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
def embed_texts(texts):
    # Returns embeddings as list of vectors
    return model.encode(texts, show_progress_bar=True)

# Prepare data and insert into ChromaDB
def store_chunks_in_chromadb(chunks, collection):
    if not chunks:
        print("No chunks to store. Skipping.")
        return
    
    texts = [chunk.text for chunk in chunks]
    if not texts:
        print("All chunks have empty text. Skipping.")
        return
    
    ids = [f"chunk_{i}" for i in range(len(texts))]
    
    # Build metadata for each chunk
    metadatas = []
    for chunk in chunks:
        if not getattr(chunk, "text", None):
            continue
        meta = {
            "headings": find_headings(chunk),
            "pdf_name": find_pdf_name(chunk),
            "page_no": find_page_number(chunk)
        }
        metadatas.append(meta)

    # Sanity check
    if len(ids) == 0 or len(texts) == 0 or len(metadatas) == 0:
        print("No valid chunks to add. Skipping.")
        return
    if not (len(ids) == len(texts) == len(metadatas)):
        print("Length mismatch! Skipping insertion.")
        return

    # Embed all texts
    embeddings = embed_texts(texts)
    
    # debug prints
    print(f"Storing {len(texts)} chunks into ChromaDB collection '{collection.name}'")

    # Add to ChromaDB collection
    collection.add(
        ids=ids,
        documents=texts,
        metadatas=metadatas,
        embeddings=embeddings
    )
    
    print(f"Inserted {len(chunks)} chunks into ChromaDB collection '{collection.name}'")

In [ ]:
client, collection = create_chromadb_collection("pdf_chunks", "./chroma_db")
store_chunks_in_chromadb(chunks, collection)

Number of chunks: 308
Number of texts: 308
Number of ids: 308
Number of metadatas: 308


Batches: 100%|██████████| 10/10 [00:05<00:00,  1.87it/s]


Inserted 308 chunks into ChromaDB collection 'pdf_chunks'
Done — your ChromaDB is persisted locally at './chroma_db'


In [ ]:
# Check if all chunks are in the database
num_chunks = collection.count()
print("Number of chunks stored in ChromaDB:", num_chunks)


Number of chunks stored in ChromaDB: 308


### RAG output
Compare the query to the documents and retrieve top k=10 documents

In [ ]:
def query_top_k(collection, query_text, k=10):
    query_embedding = model.encode([query_text])[0]

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k,
        include=["documents", "metadatas", "embeddings", "distances"]  # distances = similarity scores
    )

    top_results = []
    for doc, meta, score in zip(
        results["documents"][0],
        results["metadatas"][0],
        results["distances"][0]
    ):
        top_results.append({
            "text": doc,
            "metadata": meta,
            "score": score  # Cosine similarity (or distance depending on Chroma config)
        })

    return top_results


In [236]:
query = "Hoe zet de BBB in op cultuur?"
top_chunks = query_top_k(collection, query, k=10)

for i, chunk in enumerate(top_chunks, start=1):
    heading = chunk["metadata"]["headings"]
    output = f"## {heading} \n \n {chunk_to_markdown(chunk['text'])} \n\n **Source** \n\n Document: {chunk['metadata']['pdf_name']} \n\n Page: {chunk['metadata']['page_no']} \n\n **Ranking** \n\n Rank: {i} \n\n Score: {chunk['score']} \n ***"
    display(Markdown(output))



## Behoud het recht op contant geld 
 
 Op financieel gebied waarschuwt BBB voor het verdwijnen van contant geld. Vooral ouderen, onder­ nemers en kwetsbare groepen zijn afhankelijk van cash. Toch is contant geld bij minder dan 20 procent van de transacties nog gangbaar. BBB wil het recht op contant betalen behouden ­ voor keuzevrijheid, zelfstandigheid en privacy. 

 **Source** 

 Document: BBB.pdf 

 Page: 12 

 **Ranking** 

 Rank: 1 

 Score: 0.8746649026870728 
 ***

## Cultuur en media: geworteld in gemeenschappen, dichtbij mensen 
 
 BBB kiest voor kunst, cultuur en media die zijn geworteld in lokale, regionale en landelijke tradities en gemeenschappen. We willen een samenleving waarin verenigingen, musea, dialecten en tradities niet worden weggedrukt door de blik vanuit de Randstad. BBB wil een pluriform omroepbestel, met vitale en onafhankelijke omroepen die voor hun streek of achterban herkenbare programma's maken. Binnen het publieke omroepbestel ligt nu te veel nadruk op 'Hilversum' en zijn de oude wortels in gemeenschappen verwaarloosd. De publieke omroep moet worden hervormd, met minder bestuurslagen, meer focus op journalistieke kwaliteit en betere aansluiting bij hoe mensen vandaag media gebruiken. Kunst, cultuur en musea zijn toegankelijk voor iedereen, ongeacht waar je woont of hoeveel je verdient. Juist in de regio's buiten het zicht van Den Haag heeft BBB concrete resultaten geboekt. Cultuurgeld wordt eerlijker verdeeld over het land. Regionale en lokale omroepen worden versterkt. 

 **Source** 

 Document: BBB.pdf 

 Page: 110 

 **Ranking** 

 Rank: 2 

 Score: 0.8880108594894409 
 ***

## 15.1 Kunst en cultuur geworteld en behouden in gemeenschap en streek 
 
 Cultuur leeft in mensen, gemeenschappen en tradities. Niet in beleidsnota's en Randstedelijke top­ down subsidies. BBB zet in op cultuur van onderop: van fanfares tot dialectpoëzie, van bloemencorso's tot streekmusea. We willen jongeren weer in aanraking brengen met ambacht, erfgoed en lokale geschiedenis. Kunst en cultuur moeten inspireren, verbinden en herkenbaar zijn. 

 **Source** 

 Document: BBB.pdf 

 Page: 110 

 **Ranking** 

 Rank: 3 

 Score: 0.8928549885749817 
 ***

## 5.2 Asiel: instroom en verblijf 
 
 Er zijn grenzen aan hoeveel migranten Nederland kan opvangen. Daarom hecht BBB waarde aan een strenge selectie aan de grens. We hebben de humanitaire plicht om mensen die werkelijk op de vlucht zijn en acuut gevaar lopen, te beschermen. Echter, dit kan én hoeft niet altijd op Nederlands grond­ gebied. BBB is voorstander van zoveel mogelijk opvang in eigen regio. Grip op asielmigratie krijgen we alleen als we de regie nemen, in plaats van ons te laten overvallen. BBB kiest voor strenge, maar rechtvaardige maatregelen om de instroom daadwerkelijk te beperken. Want dweilen doe je niet met de kraan open. 

 **Source** 

 Document: BBB.pdf 

 Page: 50 

 **Ranking** 

 Rank: 4 

 Score: 0.9413849115371704 
 ***

## Waarden als fundament van onze samenleving 
 
 Daarom staat BBB pal voor onze Nederlandse cultuur en westerse waarden. Waarden zoals vrijheid, gelijkwaardigheid, verantwoordelijkheid en respect voor elkaar. Van Koningsdag tot Dodenherdenking, van dialect tot volkslied: onze tradities zijn geen achterhaalde folklore, maar bouwstenen van een gedeelde cultuur. Een cultuur die verbindt en richting geeft. 

 **Source** 

 Document: BBB.pdf 

 Page: 8 

 **Ranking** 

 Rank: 5 

 Score: 0.9718984365463257 
 ***

## Soevereiniteit voorop: niet alles uit Brussel 
 
 Veel regels die het dagelijks leven beïnvloeden komen uit Brussel. BBB is niet tegen Europese samen­ werking, maar Nederland moet weer soeverein kunnen kiezen wat werkt. Nationale belangen moeten voorop staan, niet Brusselse dogma's. 

 **Source** 

 Document: BBB.pdf 

 Page: 12 

 **Ranking** 

 Rank: 6 

 Score: 0.9815040230751038 
 ***

## Wie hier woont, draagt bij 
 
 Bij een gedeelde cultuur hoort ook gedeelde verantwoordelijkheid. Wie in Nederland wil wonen, hoort mee te doen. Inburgeren, de taal leren, werk zoeken en bijdragen aan de samenleving. Dat is geen keus, maar een voorwaarde. Wie hier wil blijven, moet hier willen zijn ­ en zich ook zo gedragen. Onze normen en waarden zijn niet onderhandelbaar. Wij zijn gastvrij, maar niet grenzeloos. Wie zich thuis voelt in Nederland, hoort daar ook verantwoordelijkheid bij te nemen. 

 **Source** 

 Document: BBB.pdf 

 Page: 8 

 **Ranking** 

 Rank: 7 

 Score: 0.9872360229492188 
 ***

## BBB wil: 
 
 - Betaalbare huur zonder verhuurders weg te jagen. BBB wil betaalbare huur zonder verhuurders weg te jagen, wat we zien gebeuren door de barrière die de Wet betaalbare huur opwerpt. We willen de Wet betaalbare huur dan ook afschaffen omdat deze leidt tot het aanzienlijk verminderen van huurwoningen. Want veel particuliere verhuurders verkopen hun woningen, mede door strengere regels. BBB onderzoekt hoe we dit kunnen keren. We willen regels aanpassen zodat verhuurders woningen blijven aanbieden in plaats van verkopen. Denk aan eerlijkere puntentellingen voor huurprijzen (woningwaardering), bijvoorbeeld geen strafpunten meer voor het ontbreken van buitenruimte. Ook willen we meer ruimte voor tijdelijke huurcontracten voor studenten. Belastingen voor (kleine) verhuurders moeten eerlijk en voorspelbaar zijn. Zo houden we de huurmarkt stabiel én betaalbaar.
- Betere ondersteuning duurzaamheid. Het aansluiten van woningen op een warmtekrachtkoppeling (WKK) is een mogelijkheid om gelijktijdig warmte en elektriciteit op te wekken voor verwarming en stroomvoorziening. Aansluiting op het net, warmtepompen of warmtenetten mag niet leiden tot een grote kostenverhoging, obstructie of vertraging van nieuwbouw. Hier zijn innovatieve oplossingen nodig. BBB zet in op duidelijke regels, minder onzekerheid en betere ondersteuning bij verduurzaming.
- Schimmelwoningen voorkomen. Veel mensen wonen in huurhuizen met schimmel. Dat is slecht voor de gezond  heid en het komt nog steeds te vaak voor. Verhuurders dienen hun verantwoorde­ lijkheid te nemen om dit probleem goed aan te pakken. BBB wil dat de overheid hierover duide­ lijke afspraken maakt met woningcorporaties, zodat woningen veilig en gezond zijn voor iedereen.
- Ondersteuning bij funderingsproblemen. Woningeigenaren horen op tijd te weten of hun funde­ ring slecht is, door steun voor onderzoek en herstelplannen. BBB onderzoekt hoe funderings­ problemen bij woningen het beste kunnen worden aangepakt, vooral in gebieden waar veel huizen verzakken door droogte of slechte grond. We kijken naar een eerlijke en gerichte manier van ondersteunen. Woningeigenaren die het zelf niet kunnen betalen, moeten worden geholpen. 

 **Source** 

 Document: BBB.pdf 

 Page: 45 

 **Ranking** 

 Rank: 8 

 Score: 0.987896203994751 
 ***

## Waarom BBB meedoet aan de CPB-doorrekening 
 
 BBB staat voor realistisch, degelijk en toekomstgericht financieel beleid. Daarom kiest BBB er bewust voor om deel te nemen aan de doorrekening van het verkiezingsprogramma door het Centraal Plan­ bureau (CPB). Wij geloven in de kracht van cijfers, transparantie en financiële onderbouwing. Tegelijkertijd erkennen we dat de CPB­modellen op onderdelen tekortschieten: ze zijn vaak te abstract, houden onvoldoende rekening met uitvoerbaarheid en missen het zicht op bredere maatschappelijke opbrengsten. Maar juist daarom is het belangrijk dat we laten zien: onze plannen houden ook binnen de officiële reken­ regels stand. 

 **Source** 

 Document: BBB.pdf 

 Page: 121 

 **Ranking** 

 Rank: 9 

 Score: 1.0091552734375 
 ***

## 3.1 Voedselzekerheid 
 
 Een stabiele en betaalbare voedselvoorziening begint bij een sterke, toekomstbestendige landbouw­ en visserijsector. Daarom staat voedselzekerheid voortaan centraal in beleid, met een eigen directie op het ministerie en een stevig afwegingskader bij ruimtelijke keuzes. BBB kiest voor realistisch beleid zonder gedwongen maatregelen, met ruimte voor boeren, vissers en voedselproducenten om te blijven doen waar ze goed in zijn. 

 **Source** 

 Document: BBB.pdf 

 Page: 33 

 **Ranking** 

 Rank: 10 

 Score: 1.0194987058639526 
 ***